# Approaches to Solve the Bellman Equation

- We have established the basic setup for Reinforcement Learning by introducing the equilibrium equations of the State-Value and Action Value functions of the Bellman Equation

- Now we look into **how** the Bellman Equations can be solved in practise

\begin{aligned}
    
    &\text{State-Value Function:} \\
    &\quad v_{\pi}(s) = \sum_a \pi(a|s) \sum_{s'} P(s' | s, a) [R(s, a, s') + \gamma v_{\pi}(s')] \\ \\

    &\text{Action-Value Function:} \\
    &\quad q_{\pi}(s,a) = \sum_{s'} P(s' | s, a) [R(s, a, s') + \gamma \sum_{a'} \pi(a'|s') q_{\pi}(s', a')]

\end{aligned}

## Overview of Methods

- In RL, there are 2 types of tasks; **prediction**, and **control**
    - **Prediction (Policy Evaluation)**: Given a policy $\pi$, estimate the state value function $V_{\pi}(s)$ and/or the action value function $Q_{\pi}(s,a)$
    - **Control (Policy Optimisation)**: Find the optimal policy $\pi^*$ 

- Generally, given a choice to estimate $Q_{\pi}(s,a)$ or $V_{\pi}(s)$, it is preferable to estimate $Q_{\pi}(s,a)$, because state values can be derived by averaging across the corresponding action values. Nontheless, there are techniques where we estimate state-value alongside action-value; say, in DP where we already know the transition probabilities and rewards.

- In this noteboook, we'll go through a few methods used in RL, and we'll see what they solve, and how they finally use this to derive $\pi^*$

- Methods
    - **Dynamic Programming**: Since we know both transition probabilities $P$ and rewards $R$, we can optimise for $\pi^*$ directly until some convergence criteria is met, because with a fixed $P$ and $R$, there is a "right answer" for $\pi^*$ with no uncertainty

    - **Monte Carlo**: When we don't have the model (no $P$ or $R$), we learn from experience by running complete episodes. 
        - We solve for the **action-value function** $Q(s,a)$ by averaging the returns observed when taking action $a$ in state $s$. 
        - The policy $\pi^*$ is then derived by acting greedily (or $\epsilon$-greedy) with respect to $Q$: $$\pi^*(s) = \arg\max_a Q(s,a)$$
        - Unlike DP which learns state values $V$ and needs the model to extract actions, MC learns $Q$ directly which is immediately actionable
    
    - **Temporal Difference**: Also model-free, but unlike MC which waits for full episodes, TD methods update estimates incrementally after each step using bootstrapping. 
        - For control tasks, TD methods like Q-learning and SARSA learn $Q(s,a)$ directly, updating via $$Q(s,a) \leftarrow Q(s,a) + \alpha[r + \gamma \max_{a'} Q(s',a') - Q(s,a)]$$. 
        - The policy is again extracted greedily from $Q_{\pi}(s,a)$. 
        - TD can also learn state values $V$ for prediction tasks, but $V$ alone is insufficient for control without the model 

    - **Policy Gradient – Direct Optimization**: When the goal is to optimize the policy directly without relying on value functions, **policy gradient methods** parameterize the policy as $\pi_\theta(a|s)$ with parameters $\theta$. 
        - The objective is to maximize the expected return $J(\theta) = \mathbb{E}[G | \pi_\theta]$. 
        - Using the **policy gradient theorem**, we can compute $\nabla_\theta J(\theta)$ and update the policy via gradient ascent:  
            $$\theta \leftarrow \theta + \alpha \nabla_\theta J(\theta)$$
        - Advantages: directly targets $\pi^*$, naturally handles stochastic policies, and works for high-dimensional or continuous action spaces. 
        - Drawbacks: typically higher variance in updates and slower convergence than value-based methods.  

    - **Actor–Critic Overview**: Combines **value-based** and **policy-based** approaches. The **actor** represents the policy $\pi_\theta(a|s)$ and is updated in the direction suggested by the **critic**, which estimates a value function $V(s)$ or $Q(s,a)$. 
        - The critic provides a low-variance estimate of the **advantage**:  $$A(s,a) = Q(s,a) - V(s)$$
        - The actor is updated using this advantage to push probabilities toward better-than-expected actions, while the critic is trained with TD methods. 
        - This setup stabilizes training compared to pure policy gradients and allows learning in continuous or large action spaces efficiently.

## Dynamic Programming (DP) - Model Based

- The simplest case of solving the Bellman Equation is when you have **both** the transition probabilities $P(s'|s,a)$ and the rewards $R(s,a,s')$

- That is, you know the full Markov Decision Process (MDP). Hence the term "model based", because we are using the MDP model

- This is, of course, exceedingly rare. But it is instructive to see how to solve for $\pi^*$ when you have almost all available information

- We will go through 2 ways of implementing the DP solver
    - Policy Interation
    - Value Iteration

- To do this, let's assume the following set up:

In [29]:
import numpy as np

np.random.seed(123)

N_STATES = 20
N_ACTIONS = 20
GAMMA = 0.9

# Shape corresponding to State, Action, Next State
TRANSITION_PROBABILITIES = np.random.rand(N_STATES, N_ACTIONS, N_STATES)
# Normalise so transition probabilities for a given state sums to 1 across all actions
TRANSITION_PROBABILITIES /= np.sum(TRANSITION_PROBABILITIES, axis=2, keepdims=True)

REWARDS = np.random.rand(N_STATES, N_ACTIONS, N_STATES) * 5

### Policy Iteration

- In policy iteration, we create a loop that does (i) policy evaluation, and (ii) policy improvement based on the evaluated outcomes of the policy, and the loop runs until some stopping criteria is reached

- **Policy Evaluation:** 
    - Since we know the transition probabilities and rewards with full certainty, we can update the State-Value function by computing the Bellman Equation
    
    - Start with a seed array representing the State-Value function $V$, and a policy $\pi$ that deterministically tells us what action to take in a given state $S$
    
    - Compute 1 pass of the Bellman State Value function update for all states. This is just a transition-probability-weighted average of the state values of the next states $S'$

    - Check that the largest change in the state value estimates for all $N$ states exceeds some threshold $\theta$. If even the largest change falls below the threshold, we assume convergence, and we return the state values $V$

    - Otherwise, keep looping

- **Policy Improvement**
    - We've previously updated our estimate of the State-Value function $V$
    - Now, we need to update our policy based on the new estimate of the State-Value function. That is, given a state $S$, I want to know the value of taking an action $A$. 
    - That is; the **Q-Value**

    - Looping over every state $S$, we init an array to hold the new Q values

    - Given $S$, loop over every action available, and update the Action-Value function $Q$. This is simply the transition-probability-weighted average of rewards from the transition $S -> S'$, plus the discounted state-value of $S'$

    - Finally, the new policy for state $S$ is simply to take the argmax of all the Q values

- **Policy Iteration**
    - Bringing these 2 together, we simply create an infinite loop of `Evaluation --> Improvement --> Evalution ...`

    - This will keep going until some convergence criteria is met. An easy on is to say that; if for some $N$ consecutive loops we do not change our policy, we have reached convergence

In [ ]:
def policy_evaluation(
    transition_probabilities: np.ndarray,
    rewards: np.ndarray,
    curr_state_value_function: np.ndarray,
    policy: np.ndarray,
    gamma: float = 0.9,
    theta: float = 1e-6
) -> np.ndarray:
    updated_state_value_function = curr_state_value_function.copy()
    while True:
        delta = 0
        for curr_state in range(N_STATES):
            v_updated = sum([
                transition_probabilities[curr_state, policy[curr_state], s_prime] * 
                (rewards[curr_state, policy[curr_state], s_prime] + gamma * updated_state_value_function[s_prime])
                for s_prime in range(N_STATES)
            ])
            delta = max(delta, abs(v_updated - updated_state_value_function[curr_state]))
            updated_state_value_function[curr_state] = v_updated
        if delta < theta:
            break
    
    return updated_state_value_function

def policy_improvement(
    transition_probabilities: np.ndarray,
    rewards: np.ndarray,
    curr_state_value_function: np.ndarray,
    curr_action_value_function: np.ndarray,
    curr_policy: np.ndarray,
    gamma: float = 0.9
) -> tuple[np.ndarray, np.ndarray]:
    updated_policy = curr_policy.copy()
    updated_action_value_function = curr_action_value_function.copy()
    for s in range(N_STATES):
        action_value_function_for_state_s = updated_action_value_function[s].copy()
        for a in range(N_ACTIONS):
            new_action_value = sum([
                transition_probabilities[s, a, s_prime] * 
                (rewards[s, a, s_prime] + gamma * curr_state_value_function[s_prime])
                for s_prime in range(N_STATES)
            ])
            action_value_function_for_state_s[a] = new_action_value
        
        updated_policy[s] = np.argmax(action_value_function_for_state_s)
        updated_action_value_function[s] = action_value_function_for_state_s
    return updated_policy, updated_action_value_function

def policy_iteration(
    transition_probabilities: np.ndarray,
    rewards: np.ndarray,
    gamma: float = 0.9,
    theta: float = 1e-6
):
    curr_state_value_function: np.ndarray = np.zeros(N_STATES)
    curr_action_value_function: np.ndarray = np.zeros((N_STATES, N_ACTIONS))
    curr_policy: np.ndarray = np.zeros(N_STATES).astype(int)
    
    count_no_change = 0
    count_iters = 0
    while True:
        print(count_iters)
        updated_state_value_function = policy_evaluation(
            transition_probabilities,
            rewards,
            curr_state_value_function,
            curr_policy,
            gamma,
            theta
        )
        updated_policy, updated_action_value_function = policy_improvement(
            transition_probabilities,
            rewards,
            updated_state_value_function,
            curr_action_value_function,
            curr_policy,
            gamma
        )
        if np.array_equal(curr_policy, updated_policy):
            count_no_change += 1
            if count_no_change >= 5:
                break
        else:
            count_no_change = 0

        curr_policy = updated_policy.copy()
        curr_action_value_function = updated_action_value_function.copy()
        curr_state_value_function = updated_state_value_function.copy()

        count_iters += 1
    
    return curr_policy, curr_action_value_function, curr_state_value_function

pi_star, q_star, v_star = policy_iteration(TRANSITION_PROBABILITIES, REWARDS)

print(f"""
    Optimal Policy: {pi_star}
    Action-Value: {q_star}
    State-Value: {v_star}
""")

0
1
2
3
4
5
6

    Optimal Policy: [12 16 18 19 17  4 16  6 19 17 17  2 10 14 18 19 17  5  7  9]
    Action-Value: [[30.94841478 31.32490423 32.19788999 31.9444741  31.79106206 31.84517046
  31.290961   31.28625888 31.67130119 31.75888851 31.64210072 31.40075868
  32.21000994 31.70431886 32.00087219 30.9218291  31.23823039 31.00114137
  31.35150682 31.38025686]
 [31.09281456 31.19694478 31.74030897 31.97479618 31.94372851 31.00817106
  31.06199313 31.43872014 31.59123965 31.17103483 31.83600553 31.7249459
  31.14808322 31.37639636 31.84617734 30.88066009 32.05620933 31.25515259
  31.48655157 31.54256258]
 [31.61237654 31.29259542 31.17586312 32.2678108  32.01706964 31.44388102
  32.08529495 31.07045071 31.49056397 31.71061504 31.02807655 30.90594336
  31.68019248 31.24952417 31.73166221 31.61723427 32.19383784 31.65359456
  32.53945021 31.81073227]
 [32.05617899 31.82333462 31.68052992 31.05107583 32.03523231 31.60465759
  31.81101442 32.20996754 31.45421427 31.10691636 31.48853139 31.

- To ascertain if your policy iteration has converged correctly, check that the action values computed from the current transition probability, rewards, and optimal state values matches the values in your optimal q value array 

In [ ]:
for s in range(N_STATES):
    for a in range(N_ACTIONS):
        q = sum([
            TRANSITION_PROBABILITIES[s, a, s_prime] *
            (REWARDS[s, a, s_prime] + GAMMA * v_star[s_prime])
            for s_prime in range(N_STATES)
        ])
        assert np.isclose(q, q_star[s, a])

### Value Iteration

- In policy iteration, notice we have 2 distinct steps
    - There is policy evaluation, which exhaustively uses given policy $\pi$ to update the state value function $v$ until convergence
    - Following which there is policy improvement, which uses the updated state value function $v$ to pick the best action $a$ for every state $s$

- The key difference between policy iteration and value iteraton is in the order we perform these steps. The methods are exactly the same, For value iteration:
    - We don't maintain an explicit policy. Instead, we focus on updating the state-value function $v$
    - The action taken at each time step is simply the maximum of the action values computed using the current $v$

- Note that there is no performance difference expected on average, and both approaches should converge to the same solution

- TLDR; 
    - Policy Iteration does Loop(evaluate policy --> improve policy)
    - Value Iteration does Loop(update state value function) 

In [ ]:
def value_iteration(
    transition_probabilities: np.ndarray,
    rewards: np.ndarray,
    gamma: float = 0.9,
    theta: float = 1e-6
) -> np.ndarray:
    curr_state_value_function = np.zeros(N_STATES)
    updated_state_value_function = np.zeros(N_STATES)
    while True:
        delta = 0
        for s in range(N_STATES):
            new_action_values = [
                sum([
                    transition_probabilities[s, a, s_prime] * 
                    (rewards[s, a, s_prime] + gamma * curr_state_value_function[s_prime])
                    for s_prime in range(N_STATES)
                ])
                for a in range(N_ACTIONS)
            ]
            updated_state_value_function[s] = max(new_action_values)
            delta = max(delta, abs(curr_state_value_function[s] - updated_state_value_function[s]))
        if delta < theta:
            break
            
        curr_state_value_function = updated_state_value_function.copy()
    
    policy = np.zeros(N_STATES).astype(int)
    for s in range(N_STATES):
        action_values = [
            sum(transition_probabilities[s, a, s_prime] * 
            (rewards[s, a, s_prime] + gamma * updated_state_value_function[s_prime])
            for s_prime in range(N_STATES))
            for a in range(N_ACTIONS)
        ]
        policy[s] = np.argmax(action_values)
    
    return policy

In [43]:
value_iteration(TRANSITION_PROBABILITIES, REWARDS)

(array([12, 16, 18, 19, 17,  4, 16,  6, 19, 17, 17,  2, 10, 14, 18, 19, 17,
         5,  7,  9]),
 array([32.21000379, 32.0562032 , 32.53944408, 32.33846026, 32.17409677,
        32.48933021, 32.14463111, 32.2494927 , 32.26843013, 32.42455566,
        32.26554727, 32.31171317, 32.43795555, 32.0959939 , 32.64112159,
        32.31968229, 32.60391839, 32.1939142 , 32.53185021, 32.59173977]))

## Monte Carlo - Sampling-Based

- We've covered the DP approach, which looks at the fairly restrictive case of solving the Bellman Equations when the full MDP is known; i.e. you know both the transition probability, and the reward. This is typically unrealistic. Very few process in reality will be kind enough to provide you both pieces of information perfectly. And even if you estimate both using data, it will be estimated under some uncertainty

- Therefore, we need more general methods to solve for the optimal policy when the transition probabilities and rewards are not known

- Here, we consider the Monte-Carlo approach. 

- Note that the traditional Monte-Carlo approach assumes an **episodic task**. That is, the iteration will run for some time, and reach a terminal point.
    - You can manually impose a terminal point for Monte-Carlo, but note that this isn't a true Monte Carlo approach, but a **truncated** Monte Ccarlo

- Idea:
    - The idea of the monte carlo is to simply let entire episodes play out multiple times, to find the long term average state-value function and hence the optimal policy
    - So you have a `step()` method that determines 2 things; (i) what is the next state you are in, and (ii) is this next state a terminal one?
    - Having done this, you now have 2 series recorded (i) a series of states, ending with a terminal state, and (ii) a series of rewards, indicating the reward you get for landing on that state
        - Note that the series of rewards is NOT the sum of your future rewards (i.e. not the true Q-value). It is the specific reward of reaching some state $S$
    - Now, for every point we land on in our array, we want to compute the discounted future rewards observed from that point, so we can derive a proper state value
    - Init an array to hold the state-value function $V$
    - We'll compute this in reverse order of the rewards array;
        - Init $G$ as the running total of the discounted rewards. This starts at 0
        - We know that the last state is terminal. Suppose this is the rightmost terminal state. 
            - We know that reaching this state gives a reward of 1. 
            - We also know that, this being the terminal state, there is no future reward. 
            - So the cumulative future reward $G = 1$
            - Since we know the rightmost state $N$ has value $G=1$, update its state-value function $V[N]$ by taking $V[N] + \alpha[G - V[N]]$
        - Moving to the second last state. 
            - We know that this state has reward 0. Therefore, the value of this state comes only from the discounted value of transitioning to the last state and receiving the terminal reward 1. 
            - Thus, update $G = reward[N-1] + \gamma \cdot G$
            - Then, update the state value function $V[N-1] + \alpha[G - V[N-1]]$
        - Iterate until you reach the end of the episode 
    - Continue to the next episode for a fixed number of episodes

In [ ]:
import numpy as np
np.random.seed(0)

# Environment setup; we create a number line with 10 states. The right-most state has a reward of 1
# Therefore, train the RL loop to take rightward actions [+1] more than leftward actions [-1]
# Terminate if you reach the left or rightmost states
N_STATES = 5
N_ACTIONS = 5
GAMMA = 0.9
ALPHA = 0.1
EPSILON = 0.5
EPISODES = 1000
TD_N = 3

TRANSITION_PROBABILITIES = np.random.rand(N_STATES, N_ACTIONS, N_STATES)
TRANSITION_PROBABILITIES /= np.sum(TRANSITION_PROBABILITIES, axis=2, keepdims=True)
REWARDS = np.random.rand(N_STATES, N_ACTIONS, N_STATES) * 5

In [ ]:
from collections import namedtuple

episode_step = namedtuple('episode_step', ['curr_state', 'action', 'new_state', 'reward'])

def take_action(curr_state: int, action_values: np.ndarray) -> tuple[int]:
    action_values_for_curr_state = action_values[curr_state, :]
    
    if np.random.rand() >= EPSILON:
        highest_action_value_action = np.argmax(action_values_for_curr_state)
        return highest_action_value_action
    else:
        random_action = np.random.choice(N_ACTIONS)
        return random_action

def monte_carlo_epsilon_greedy():
    action_values = np.zeros((N_STATES, N_ACTIONS))
    state_values = np.zeros(N_STATES)

    rewards_intermediate_store = {
        (s,a): [] for s in range(N_STATES) for a in range(N_ACTIONS)
    }
    for _ in range(EPISODES):
        curr_state = np.random.choice(list(range(1,N_STATES-1)))
        episode_history = []

        terminal_state_reached = curr_state in [0, N_STATES-1]
        while not terminal_state_reached:
            action = take_action(curr_state, action_values)
            new_state = np.random.choice(
                N_STATES, p=TRANSITION_PROBABILITIES[curr_state, action]
            )
            terminal_state_reached = new_state in [0, N_STATES-1]
            reward = REWARDS[curr_state, action, new_state]
            episode_history.append(
                episode_step(curr_state=curr_state, action=action, new_state=new_state, reward=reward)
            )
            curr_state = new_state
        
        return_val = 0
        for step in reversed(episode_history):
            return_val = step.reward + GAMMA * return_val
            rewards_intermediate_store[(step.curr_state, step.action)].append(return_val)
        
        for curr_state, action in rewards_intermediate_store.keys():
            action_values[curr_state, action] = np.mean(rewards_intermediate_store.get((curr_state, action))) if rewards_intermediate_store.get((curr_state, action)) != [] else 0
        
        state_values = np.max(action_values, axis=1)
    
    return state_values, action_values


In [246]:
monte_carlo_epsilon_greedy()

(array([ 0.        , 11.06340833,  9.65237593, 11.93171671,  0.        ]),
 array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 7.31191006, 11.06340833,  9.40697801,  7.43068361,  5.76089819],
        [ 8.37778159,  9.59896629,  9.65237593,  8.6572607 ,  7.6457071 ],
        [ 9.9728158 ,  8.30667043,  7.33269837,  9.40128791, 11.93171671],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]]))

## Temporal-Difference (TD) - Bootstrapped Learning

- The motivation for this is similar to the Monte Carlo approach; this is used when we can't solve the Bellman Equation because transition probabilities and/or rewards are not known

- Unlike Monte Carlo, which relies on the assumption that the task is episodic (i.e. terminates at some point), TD lets you update state value functions and policy incrementally. Therefore, TD is applied to tasks with no natural termination state

- We'll cover 2 forms of TD;
    - **TD(N)**: N-Step TD Learning
    - **TD($\lambda$)**: $\lambda$ Return TD Learning

### N-Step TD

- In the N-Step TD, we want to update either the state value function $V(S_t)$ or action-value function $Q(S_t, a)$ based on some observed reward plus the estimated value of the next state.
    - The $N$ simply tells you how many observed rewards we use in updating our current state value $V(S_t)$

- TD doesn't dictate whether you should learn $Q$ or $V$; it simply gives an approach to learn these by updating an estimate using the difference between successive estimates; or the **TD Error**. This is defined as:

\begin{aligned}
    \delta_t &= R_t + \gamma V(S_{t+1}) - V(S_t)
\end{aligned}

- For example, in `TD(0)`, the updated state-value is the current state-value plus some learning rate multipled by the observed deviation from your current state-value. The observed deviation is the return you experience $R_t + \gamma V(S_{t+1})$ minus the current estimated expected return $V(S_t)$
\begin{aligned}
    V(S_t) &= V(S_t) + \alpha (\hat{G_t^{(1)}} - V(S_t)) \\
    &= V(S_t) + \alpha (R_t + \gamma V(S_{t+1}) - V(S_t))
\end{aligned}
    - Here, we update $V(S_t)$ at every step. So $G_t^{(1)}$ is not known, but your best guess at the time step.
    - This is why some tutorials mention that `TD(N)` approach requires bootstrapping; it comes from the fact that $\hat{G_t^{(1)}} = R_t + \gamma V(S_{t+1})$ is an estimate based on $V(S_{t+1})$
    - This is known as the **1-step bootstrap return**
    - This differs from something like Monte Carlo, for example, where episodes are played out till the end, and we do not need to bootstrap because we use the actual rewards $R_t, R_{t+1}, ...$ to compute our state-value function instead of some intermediate state-value estimate $V(S_{t+1})$

- Extending this idea, for `TD(N)`, we accumulate $N$ steps of the future rewards $R_t, R_{t+1}, ... R_{t+n}$ plus the estimated state value $V(S_{t+n})$ to compute the **TD error** 
\begin{aligned}
    V(S_t) &= V(S_t) + \alpha (\hat{G_t^{(N)}} - V(S_t)) \\
    &= V(S_t) + \alpha (\sum_{i=0}^{n-1} \gamma^{i} R_{t+i} +  \gamma^n V(S_{t+n}) - V(S_t))
\end{aligned}
    - Similar to the example above, $\hat{G_t^{(N)}}$ is known as the **N-step bootstrap return**
    - Fun fact; if N is the episode length, then this is simply Monte Carlo!

- The intuition here is: if $V(S_t)$ has converged, then there should be no TD error observed, the current step return should equal the current state values. So $V(S_t)$ should be unchanged

- Note that the $TD(N)$ idea can be applied to both **continuing** and **episodic** tasks. For clarity, we will implement both here

In [74]:
import numpy as np
np.random.seed(0)

# Environment setup; we create a number line with 10 states. The right-most state has a reward of 1
# Therefore, train the RL loop to take rightward actions [+1] more than leftward actions [-1]
N_STATES = 10
N_ACTIONS = 5
GAMMA = 0.9
ALPHA = 0.1
EPSILON = 0.5
EPISODES = 1000
TD_N = 5

N_STEPS = 10

TRANSITION_PROBABILITIES = np.random.rand(N_STATES, N_ACTIONS, N_STATES)
TRANSITION_PROBABILITIES /= np.sum(TRANSITION_PROBABILITIES, axis=2, keepdims=True)
REWARDS = np.random.rand(N_STATES, N_ACTIONS, N_STATES) * 5

TERMINAL_STATES = [0, N_STATES-1]

In [ ]:
import numpy as np
from collections import namedtuple

episode_step = namedtuple('episode_step', ['state', 'reward', 'next_state'])

def update_timestep_is_valid(timestep_to_update: int) -> bool:
    return timestep_to_update >= 0

def next_state_is_terminal(next_state: int) -> bool:
    return next_state in [0, N_STATES-1]

def bootstrap_state_is_nonterminal(update_td_return_for_this_timestep, reward_steps_available, termination_time):
    bootstrap_state_index = update_td_return_for_this_timestep + reward_steps_available
    return bootstrap_state_index < termination_time

def n_step_td_episodic(TD_N: int):
    ## Init array to store state-values
    V = np.zeros(N_STATES)

    ## Going across all `EPISODES`
    for _ in range(EPISODES):

        ## Init current state such that it is not a terminal state
        curr_state = np.random.randint(1, N_STATES - 1)
        
        ## For `TD_N`, we need to store N steps of state updates in the trajectory
        trajectory = []
        
        ## For every episode, we define a variable to hold termination time. 
        ## This will be populated in the loop
        termination_time = np.inf
        
        ## Set current episode time as 0
        curr_time = 0  

        ## Playing through the whole episode...
        while True:

            ## If curr_time is less than termination time, we make ONE transition
            if curr_time < termination_time:

                ## Pick an action
                curr_action = np.random.randint(N_ACTIONS)

                ## Make a transition
                next_state = np.random.choice(N_STATES, p=TRANSITION_PROBABILITIES[curr_state, curr_action])

                ## Get the reward of the transition
                reward = REWARDS[curr_state, curr_action, next_state]

                ## Store the transition in the trajectory history
                trajectory.append(episode_step(curr_state, reward, next_state))

                ## If we reach a terminal state
                if next_state_is_terminal(next_state):
                    ## Set termination time condition to the next time step. The condition `curr_time < termination_time` in the episode loop will therefore fail in the next step
                    termination_time = curr_time + 1 
                
                ## Set current state to the next_state
                curr_state = next_state

            # Identify which index we should update. Remember that in TD(N), we need to accumulate N steps in our buffer before updating reward. Therefore, the index we update must always be TD_N - 1 steps behind. For example, to update the first state value index seen in our episode, if TD_N = 3, we need to have made 3 transitions. This happens at curr_time = 2, where we have (0 --> 1, 1 --> 2, 2 --> 3). At this point, we want to update index at 2 - 3 + 1 = 0 
            update_td_return_for_this_timestep = curr_time - TD_N + 1

            ## Assert that the update logic is valid (i.e. your update index >= 0)
            if update_timestep_is_valid(update_td_return_for_this_timestep):
                
                ## Use `G` as the accumulator for the rewards
                G = 0.0
                
                
                ## Depending on the length of the episode, we may not be able to update using the full TD_N update. For example, let's say you want a TD(3) update. Through your simulation, you end up with a trajectory with 5 `episode_steps`, which correspond to the transitions (0 --> 1, 1 --> 2, 2 --> 3, 3 --> 4, 4 --> 5)

                ## For TD(3), we need to have 3 rewards available before updating. This happens in the third timestep. So (0 --> 1, 1 --> 2, 2 --> 3). Therefore at time index 2, we can compute the TD return for the state at time index 0, which also happens to be state 0.

                ## So this update goes on. At 4th timestep, we update time index 1. At 5th timestep, we update time index 2. Now, we reach the time to update time index 3. For this, we need (3 --> 4, 4 --> 5, 5 --> 6). But wait, since 5 state 5 at time index 5 is terminal, we don't have a 5 --> 6!! What do we do?

                ## Instead of discarding usable reward information, TD(N) will make use of as many rewards as possible. Since we cannot do TD(3) without 5 --> 6, we do a TD(2) update instead!

                ## Thus, here we keep track of the number of rewards we have available. When we reach termination, we are at time step 4, and termination_time = 5. But we want to keep updating our state value `V` even if a we don't meet the criteria for TD(3). So for fixed termination time 5, suppose we reach curr_time = 5. Then `update_td_return_for_this_timestep` becomes 5-3+1 = 3. To update index 3, we then say the number of rewards available is min(3, 5-3) = 2
                reward_steps_available = min(
                    TD_N, 
                    termination_time - update_td_return_for_this_timestep
                )

                ## For every reward available
                for i in range(int(reward_steps_available)):
                    ## Add the reward multiplied by the relevant gamma discount
                    G += (GAMMA ** i) * trajectory[update_td_return_for_this_timestep + i].reward

                ## if the time index to update + reward_steps_available is less than termination_time, it means that the next state value is available, because it is non terminal. So we can add the bootstrapped state-value 

                ## BUT if we get update_td_return_for_this_timestep + reward_steps_available == termination_time, then by definition, the last timestep must be terminal. Then there is no need to add the bootstrapped state value of the next state, because there is no "next state" after the terminal state
                if bootstrap_state_is_nonterminal(reward_steps_available=reward_steps_available, update_td_return_for_this_timestep=update_td_return_for_this_timestep, termination_time=termination_time):

                    ## After adding N rewards, add the bootstrapped state value of the next state. This only applies if we have not reached terminal state. Otherwise the value of the next state is 0 anyway, because there is no post-terminal transition
                    
                    G += (GAMMA ** reward_steps_available) * V[
                        trajectory[
                            update_td_return_for_this_timestep + reward_steps_available - 1
                        ].next_state
                    ]

                # Update state value
                state_to_update = trajectory[update_td_return_for_this_timestep].state
                V[state_to_update] += ALPHA * (G - V[state_to_update])

            curr_time += 1

            # Break out of the while loop if our update value is termination_time - 1 (because there are no further rewards after this step)
            if update_td_return_for_this_timestep == termination_time - 1:
                break

    return V


In [76]:
n_step_td_episodic(1)

array([ 0.        ,  9.65660816, 10.57941725,  9.19019031,  7.76120898,
        8.84684624,  9.45925917,  8.59510287,  9.85313675,  0.        ])

In [87]:
from collections import deque, namedtuple
import numpy as np

episode_step = namedtuple('episode_step', ['state', 'reward', 'next_state'])

def n_step_td_continuing(steps: int = N_STEPS):
    """
    N-step TD for continuing tasks.
    """

    V = np.zeros(N_STATES, dtype=float)

    # Store 
    td_n_buffer = deque(maxlen=TD_N)

    # Start at a random (non-terminal) state
    curr_state = np.random.randint(1, N_STATES - 1)

    for _ in range(steps):
        # Random action (ε-greedy or random for continuing case)
        action = np.random.randint(N_ACTIONS)

        # Sample next state using transition probabilities
        next_state = np.random.choice(N_STATES, p=TRANSITION_PROBABILITIES[curr_state, action])

        # Sample reward for (s, a, s_next)
        reward = REWARDS[curr_state, action, next_state]

        # Append to buffers
        td_n_buffer.append(episode_step(state=curr_state, reward=reward, next_state=next_state))

        # When we have enough samples, perform an update
        G = 0
        if len(td_n_buffer) == TD_N:
            for t, step in enumerate(td_n_buffer):
                G += (GAMMA ** t) * step.reward
            G += (GAMMA ** TD_N) * V[td_n_buffer[-1].next_state]  # bootstrap from S_{t+n}

            state_to_update = td_n_buffer[0].state
            V[state_to_update] += ALPHA * (G - V[state_to_update])

            # Slide window manually
            td_n_buffer.popleft()

        curr_state = next_state

    return V

In [110]:
n_step_td_continuing(20000)

array([25.2924899 , 25.06303537, 24.91916825, 24.80299663, 25.0760952 ,
       25.72068895, 25.83358932, 25.0155259 , 24.44503252, 24.915488  ])

### $\lambda$ Return TD

- Having discussed $TD(N)$, we now move on to $TD(\lambda)$. This is very similar to the $TD(N)$ case, we want to use successive TD errors to update either the state value $V(s)$ or action value $Q(s,a)$

- Recall in $TD(N)$ that we define some $N$ steps of reward accumulation before bootstrapping an update

\begin{aligned}
    G^{(n)}_t &= R_{t+1} + \gamma R_{t+2} + ... + \gamma^{n-1} R_{t+n} + \gamma^n V(S_{t+n})
\end{aligned}

- The question arises; what value of $N$ do we choose? 
    - If $N$ is small, we update fast, and therefore the variance is low, because we each step is small. But because we don't accumulate the update over multiple observations of rewards, the estimate is more biased
    - If $N$ is large, we update slowly averaging over more reward steps. Thus, the bias is lower. But since each update takes longer to accumulate, the variance is also larger

- To resolve this problem of choosing $N$, we introduce $TD(\lambda)$

- $TD(\lambda)$ uses **all** possible future steps without bootstrapping, but **exponentially weights** them using $\lambda \in [0, 1)$. The idea here is that you don't need to choose which steps are important; since you are using all of them, they are all important. You just determine how best to weight them using $\lambda$

- Formally, the $TD(\lambda)$ setup is:

\begin{aligned}
    G^{(\lambda)}_t &= (1 - \lambda) \sum_{n=1}^{\infty} \lambda^{n-1} G_t^{(n)}
\end{aligned}

- Why geometric weighting? Because geometric weights are guaranteed to sum to 1. Looking at the weights:

\begin{aligned}
    w_n &= (1 - \lambda) \sum_{n=1}^{\infty} \lambda^{n-1} \\
    &= (1 - \lambda) \sum_{n=0}^{\infty} \lambda^{n} \\
    &= (1 - \lambda) \cdot \frac{1}{1 - \lambda} & \text{by geometric series} \\
    &= 1
\end{aligned}

- Since the geometric weights only sum to 1 in the limit, $TD(\lambda)$ should conceptually only be valid for continuous learning tasks, not episodic task. In theory, you probably could apply it to episodic tasks, and lose the "weighted average" interpretation of the weights. But for the purposes of accuracy, we'll focus on the continuous learning task

#### General form of $TD(\lambda)$

- Let $G_t(N)$ be the $TD(N)$ return
\begin{aligned}
    G_t(1) &= R_{t+1} + \gamma V(S_{t+1}) \\
    G_t(2) &= R_{t+1} + \gamma R_{t+2} + \gamma^2 V(S_{t+3}) \\\
    G_t(3) &= R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \gamma^3 V(S_{t+4})
    ...
\end{aligned}

- Recall that in $TD(\lambda)$, we are simply taking the exponentially weighted sums of the $TD(N)$ returns. Therefore:

\begin{aligned}
    G_t(\lambda) &= (1 - \lambda) \cdot [\lambda^0 G_t(1) + \lambda^1 G_t(2) + \lambda^2 G_t(3) + ...] \\
    &= (1 - \lambda) \cdot [ R_{t+1} + \gamma V(S_{t+1}) + \lambda (R_{t+1} + \gamma R_{t+2} + \gamma^2 V(S_{t+2})) + \lambda^2 (R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \gamma^3 V(S_{t+3})) + ...] \\
    &= (1 - \lambda) \cdot [ (R_{t+1} + \lambda R_{t+1} + \lambda^2 R_{t+1} + ...) + (\lambda \gamma R_{t+2} + \lambda^2 \gamma R_{t+2} + ...) + (\lambda^2 \gamma^2 R_{t+3} + \lambda^3 \gamma^2 R_{t+3} + ...) + ... + (\gamma V_{s+1} + \lambda \gamma^2 V_{s+2} + \lambda^2 \gamma^3 V_{s+3} + ...)] \\
\end{aligned}

- Let's first focus on the weights $W(R_{t+k})$ of the reward terms:

\begin{aligned}
    W(R_{t+1}) &= (1 - \lambda) [1 + \lambda + \lambda^2 + ...] \\
    &= (1 - \lambda) \frac{1}{1 - \lambda} \\
    &= 1 \\ \\

    W(R_{t+2}) &= (1 - \lambda) [\lambda \gamma + \lambda^2 \gamma + \lambda^3 \gamma ...] \\
    &= (1 - \lambda) \frac{\lambda \gamma}{1 - \lambda} \\
    &= \lambda \gamma \\ \\

    W(R_{t+3}) &= (1 - \lambda) [\lambda^2 \gamma^2 + \lambda^3 \gamma^2 + \lambda^4 \gamma^2 ...] \\
    &= (1 - \lambda) \frac{\lambda^2 \gamma^2}{1 - \lambda} \\
    &= \lambda^2 \gamma^2 \\ \\
\end{aligned}

- From the pattern above, for a given $R_{t+k}$, we see that as $\lambda \rightarrow 1$, we will get $(\lambda \gamma)^{k-1} = \gamma^{k-1}$

\begin{aligned}
    W(R_{t+k}) &= (1 - \lambda) [\lambda^{k-1} \gamma^{k-1} + \lambda^{k} \gamma^{k-1} + \lambda^{k+1} \gamma^{k-1} + ...] \\
    &= (1 - \lambda) \gamma^{k-1} \sum_{i=k-1}^{\infty} \lambda^i \\
    &= (1 - \lambda) \gamma^{k-1} \frac{\lambda^{k-1}}{1 - \lambda} \\
    &= (\gamma \lambda)^{k-1} 
\end{aligned}

- Next, we focus on the weight on the bootstrap term for each return $G^{(n)}_t$, which takes the general form $W_V(S_{t+k})$. Recall that we previously accounted for all reward weights, so we collect the $1 - \lambda$ term into the weight of the bootstrap term:

\begin{aligned}
    W_V(S_{t+k}) &= (1 - \lambda) \lambda^{k-1} \gamma^{k} \\
\end{aligned}

- As such, we arrive at the key $TD(\lambda)$ formulation:

\begin{aligned}
    G_t(\lambda) &= (1 - \lambda) \cdot [\lambda^0 G_t(1) + \lambda^1 G_t(2) + \lambda^2 G_t(3) + ...] \\
    &= (1 - \lambda) \cdot [ R_{t+1} + \gamma V(S_{t+1}) + \lambda (R_{t+1} + \gamma R_{t+2} + \gamma^2 V(S_{t+2})) + \lambda^2 (R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \gamma^3 V(S_{t+3})) + ...] \\
    &= (1 - \lambda) \cdot [ (R_{t+1} + \lambda R_{t+1} + \lambda^2 R_{t+1} + ...) + (\lambda \gamma R_{t+2} + \lambda^2 \gamma R_{t+2} + ...) + (\lambda^2 \gamma^2 R_{t+3} + \lambda^3 \gamma^2 R_{t+3} + ...) + ... + (\gamma V_{s+1} + \lambda \gamma^2 V_{s+2} + \lambda^2 \gamma^3 V_{s+3} + ...)] \\
    &= \sum_{k=1}^{\infty} (\gamma \lambda)^{k-1}  R_{t+k} + (1 - \lambda) \sum_{n=1}^{\infty} \lambda^{n-1} \gamma^n V(S_{t+n})
\end{aligned}

#### Boundary values of $\lambda$

- It's useful to see what happens to the formulation at the boundary values of $\lambda$

- If $\lambda = 0$, all powers of $\lambda^{n-1} = 0$, except for the case $0^0 = 1$. So $TD(\lambda)$ reduces to a $TD(N=0)$ return
\begin{aligned}
    G^{(\lambda)}_t &= (1 - \lambda) \sum_{n=1}^{\infty} \lambda^{n-1} G_t^{(n)} \\
    &= \sum_{n=1}^{\infty} \lambda^{n-1} G_t^{(n)} \\
    &= G_t^{(1)} \\
    &= R_{t+1} + \gamma V(S_{t+1})
\end{aligned}

- If $\lambda \rightarrow 1$, in the limit, $TD(\lambda=1)$ simply becomes Monte Carlo, where we take the full episodic reward

\begin{aligned}
G_t(\lambda) &= \sum_{k=1}^{\infty} (\gamma \lambda)^{k-1}  R_{t+k} + (1 - \lambda) \sum_{n=1}^{\infty} \lambda^{n-1} \gamma^n V(S_{t+n}) \\
&= \sum_{k=1}^{\infty} (\gamma)^{k-1}  R_{t+k} \\
&= R_{t+1} + \gamma R_{t+2} + ...
\end{aligned}

#### Backward vs Forward Views

- Having gone through the theory of $TD(\lambda)$, there is an immediate gotcha when implementing - it's not possible to actually accumulate infinite reward terms to compute $G$

\begin{aligned}
    G_t(\lambda) &= (1 - \lambda) \sum_{n=1}^{\infty} \lambda^{n-1} G_t(n)
\end{aligned}

- Let's consider another view of this. Recall that in TD learning, we are trying to incrementally update some value function by the TD error:

\begin{aligned}
    V(S_t) \leftarrow V(S_t) + \alpha [G_t(\lambda) - V(S_t)]
\end{aligned}

- Let's call $G_t(\lambda) - V(S_t)$ the $\lambda$ return error

- **CLAIM:** Since we cannot compute $G_t(\lambda)$ by taking the infinite sum of $TD(N)$ terms (the **forward view**), we can replace the $\lambda$ return error with the weighted sum to infinity of one step TD errors (**backward view**):

\begin{aligned}
    G_t(\lambda) - V(S_t) &= \sum_{k=0}^{\infty} (\gamma \lambda)^{k} \delta_{t+k} \\ \\

    \delta_{t+k} &= R_{t+k} + \gamma V(S_{t+k+1}) - V(S_{t+k})
\end{aligned}

- We previously said that we cannot accumulate infinite rewards before computing $G_t(\lambda)$. But isn't this just another sum to infinity?
    - Yes, it is. So this also cuts off at some arbitrary point. 
    - But unlike summing the TD(N) terms, we don't need a buffer to accumulate rewards in this approach! 
    - Since updates are happening for every $\delta_{t+k}$, and $\delta_{t+k}$ can be computed at every step, there is no delay in the updating of the state value at time $t$, so long as we can accept bootstrapped values of the next state $V(S_{t+k+1})$

##### Proof that backward = forward view


- Let's start with definitions:

\begin{aligned}
    \delta_t &= R_{t+1} + \gamma V(S_{t+1}) - V(S_t) \\
    G_t(n) &= \sum_{j=1}^{n} \gamma^{j-1} R_{t+j} + \gamma^n V(S_{t+n}) \\
    G_t(\lambda) &= (1 - \lambda) \sum_{n=1}^{\infty} \lambda^{n-1} G_t(n) \\ 
    &= \sum_{k=1}^{\infty} (\gamma \lambda)^{k-1}  R_{t+k} + (1 - \lambda) \sum_{n=1}^{\infty} \lambda^{n-1} \gamma^n V(S_{t+n}) & \text{Forward View}
\end{aligned}

- Looking at the RHS of the equation to prove, let's call the summation of discounted 1-step TD errors $S$. So we have

\begin{aligned}
    S &= \sum_{k=0}^{\infty} (\gamma \lambda)^{k} \delta_{t+k}
\end{aligned}

- Now, by the definition above $\delta_t = R_{t+1} + \gamma V(S_{t+1}) - V(S_t)$

\begin{aligned}
    S &= \sum_{k=0}^{\infty} (\gamma \lambda)^{k} \delta_{t+k} \\
    &= \sum_{k=0}^{\infty} (\gamma \lambda)^{k} [R_{t+k+1} + \gamma V(S_{t+k+1}) - V(S_{t+k})] \\
    &= \underbrace{\sum_{k=0}^{\infty} (\gamma \lambda)^k R_{t+k+1}}_{(A)} + 
    \underbrace{\sum_{k=0}^{\infty} (\gamma \lambda)^k \gamma V(S_{t+k+1})}_{(B)} - 
    \underbrace{\sum_{k=0}^{\infty} (\gamma \lambda)^k V(S_{t+k})}_{(C)}
\end{aligned}

- Let's rewrite $(C)$ by separating out the first term of the summation, when $k=0$

\begin{aligned}
    (C) &= \sum_{k=0}^{\infty} (\gamma \lambda)^k V(S_{t+k}) \\
    &= V(S_{t+k}) + \sum_{k=1}^{\infty} (\gamma \lambda)^k V(S_{t+k}) \\
\end{aligned}

- Then, rewriting $S$

\begin{aligned}
    S &= \sum_{k=0}^{\infty} (\gamma \lambda)^{k} \delta_{t+k} \\
    &= (A) + (B) - (C) \\
    &= (A) + \underbrace{(B) - \sum_{k=1}^{\infty} (\gamma \lambda)^k V(S_{t+k})}_{(D)} - V(S_{t+k})
\end{aligned}

- Notice that the term we extracted can be combined with $(B)$, which leads to 

\begin{aligned}
    (D) &= \sum_{k=0}^{\infty} (\gamma \lambda)^k \gamma V(S_{t+k+1}) - \sum_{k=1}^{\infty} (\gamma \lambda)^k V(S_{t+k}) \\
    &= \sum_{k=1}^{\infty} (\gamma \lambda)^{k-1} \gamma V(S_{t+k}) - \sum_{k=1}^{\infty} (\gamma \lambda)^k V(S_{t+k}) \\
    &= \sum_{k=1}^{\infty} (\gamma \lambda)^{k-1} \gamma V(S_{t+k}) - (\gamma \lambda)^k V(S_{t+k}) \\
    &= \sum_{k=1}^{\infty} V(S_{t+k}) [(\gamma \lambda)^{k-1} \gamma - (\gamma \lambda)^k] \\
    &= \sum_{k=1}^{\infty} V(S_{t+k}) (\gamma \lambda)^{k-1} [\gamma - \gamma \lambda] \\
    &= \sum_{k=1}^{\infty} V(S_{t+k}) (\gamma \lambda)^{k-1} \gamma (1 - \lambda) \\
    &= (1 - \lambda) \sum_{k=1}^{\infty} \gamma^k \lambda^{k-1} V(S_{t+k}) \\
\end{aligned}

- Looking at the forward view, this is exactly the bootstrap term!

- Simiarly, let's rewrite $(A)$

\begin{aligned}
    (A) &= \sum_{k=0}^{\infty} (\gamma \lambda)^k R_{t+k+1} \\
    &= \sum_{k=1}^{\infty} (\gamma \lambda)^{k-1} R_{t+k} \\
\end{aligned}

- Going back to the forward view again, this is exactly the discounted reward term!!

- So going back to $S$

\begin{aligned}
    S &= \sum_{k=0}^{\infty} (\gamma \lambda)^{k} \delta_{t+k} \\
    &= \underbrace{\sum_{k=0}^{\infty} (\gamma \lambda)^k R_{t+k+1}}_{(A)} + 
    \underbrace{\sum_{k=0}^{\infty} (\gamma \lambda)^k \gamma V(S_{t+k+1})}_{(B)} - 
    \underbrace{\sum_{k=0}^{\infty} (\gamma \lambda)^k V(S_{t+k})}_{(C)} \\
    &= \sum_{k=1}^{\infty} (\gamma \lambda)^{k-1} R_{t+k} + (1 - \lambda) \sum_{k=1}^{\infty} \gamma^k \lambda^{k-1} V(S_{t+k}) - V(S_{t+k}) \\ \\

    &\therefore V(S_{t+k}) + S = G_{t+k}(\lambda) \\
    G_{t+k}(\lambda) &= V(S_{t+k}) + \sum_{k=0}^{\infty} (\gamma \lambda)^{k} \delta_{t+k}
\end{aligned}

- This tells us that, though the forward view asks us to accumulate $\infty$ rewards before updating the current state $t$, it is actually equivalent to taking the current state value, and adding discounted 1-step TD errors for infinite steps!

- The main difference here is that, of course, we no longer need to "buffer". The state values can be updated incrementally!

- This is why, to update state value, we can simply add the summation of 1 step TD errors to the existing state value

\begin{aligned}
    V(S_t) \leftarrow V(S_t) + \alpha [G_t(\lambda) - V(S_t)] \\
    V(S_t) \leftarrow V(S_t) + \alpha [\sum_{k=0}^{\infty} (\gamma \lambda)^{k} \delta_{t+k}] \\
\end{aligned}

- And we don't need to wait for $\infty$ steps before updating, because at every step, I simply need to attribute the 1-step TD error proportionally to every state $S_t$! 
    - That is, suppose I have states 1,2,3 and I move from 1 --> 2 --> 3
    - Then 
        - the TD error I get from step 2 will be added to V(S_2) with weight $(\gamma \lambda)^0$
        - the TD error I get from step 2 will be added to V(S_1) with weight $(\gamma \lambda)^1$

- The idea here is that at every step, I update all prior visited states. Just scaling the error by the number of timesteps ago I visited the state

- To maintain the weights, we simply need to maintain the product of $(\gamma \lambda)^x$ for every state, and add 1 for the latest state visited. Then we just multiply the entire array by a factor of $(\gamma \lambda)$ in each step
    - This is known as the **eligibility trace**


#### Implementation

In [144]:
import numpy as np
np.random.seed(0)

N_STATES = 10
N_ACTIONS = 5
GAMMA = 0.9
ALPHA = 0.1
EPISODES = 1000
TD_LAMBDA = 0.5

N_STEPS = int(1e5)

TRANSITION_PROBABILITIES = np.random.rand(N_STATES, N_ACTIONS, N_STATES)
TRANSITION_PROBABILITIES /= np.sum(TRANSITION_PROBABILITIES, axis=2, keepdims=True)
REWARDS = np.random.rand(N_STATES, N_ACTIONS, N_STATES) * 5

In [145]:
from collections import deque, namedtuple

episode_step = namedtuple('episode_step', ['curr_state', 'action', 'reward', 'next_state'])

def td_lambda_continuing():
    V = np.zeros(N_STATES)
    curr_state = np.random.randint(N_STATES)
    ELIGIBILITY_TRACE = np.zeros(N_STATES)
    for _ in range(N_STEPS):
        action = np.random.randint(N_ACTIONS)
        next_state = np.random.choice(N_STATES, p=TRANSITION_PROBABILITIES[curr_state, action])
        curr_reward = REWARDS[curr_state, action, next_state]

        one_step_td_error = curr_reward + (GAMMA * V[next_state]) - V[curr_state]

        ELIGIBILITY_TRACE[curr_state] += 1
        V[curr_state] += ALPHA * one_step_td_error
        
        ELIGIBILITY_TRACE *= TD_LAMBDA * GAMMA
        curr_state = next_state
    
    return V

td_lambda_continuing()

array([24.77539107, 24.69010724, 24.40725054, 25.26559653, 24.88448932,
       24.78805943, 24.97204458, 25.46356333, 24.68696901, 24.95584939])

## Policy Gradient – Direct Optimization

### Idea

- In **Policy Gradient** approach, we attempt to parameterize the policy $\pi$ using some set of parameters $\theta$. 

- Then, to improve our policy, we iteratively modify $\theta$ by using the first derivative of the expected reward $J_{\pi_{\theta}}$ w.r.t a change in $\theta$. That is:
\begin{aligned}
    \frac{\partial J_{\pi_{\theta}}}{\partial \theta}
\end{aligned}

- This is known as **gradient ascent**. It's an ascent, because we want to maximise the reward $J$, instead of the usual gradient descent in supervised learning when we are minimising some loss $L$

- For each iteration, we update $\theta$ by taking

\begin{aligned}
    \theta \leftarrow \theta + \alpha \cdot \bigtriangledown_{\theta} \sum_{t=0}^{T-1} \log \pi_{\theta}(a_t | s_t) \cdot G_t
\end{aligned}

### Theory: How does the gradient ascent update come about?

- In the policy gradient approach, we want to maximise the expected return $J$ under some policy $\pi_{\theta}$, where the policy is entirely dependent on parameter $\theta$

\begin{aligned}
    J(\theta) &= E_{\tau \sim \pi_{\theta}}[R(\tau)] \\
    &\text{where} \\
    &\quad \tau = [(s_0, a_0), (s_1, a_1), ...] \text{ is the trajectory of states and actions } (s_t, a_t) \\
    &\quad R(\tau) = \sum_{t=0}^{\infty} \gamma^t r(s_t, a_t) \text{ is the discounted total reward} \\
    &\quad \pi_{\theta}(a|s) \text{ is the theta-parameterised probability of taking action } a \text{ at state } s\\
\end{aligned}

- So if we want to take different actions, we need to modify $\theta$. And to decide how to modify $\theta$, we use the gradient $\frac{\partial J(\theta)}{\partial \theta}$. This will be written as $\bigtriangledown_{\theta} J(\theta)$

- Intuitively, since we want to maximise $J$, if the gradient is positive, we want to move in the direction of the gradient!

#### Computing the Gradient

- Let's rewrite the return $J(\theta)$ as the summation over all trajectories $\tau$, instead of leaving it as the expectation

\begin{aligned}
    J(\theta) &= \sum_{\tau} P(\tau; \theta) R(\tau)
\end{aligned}

- We know that $P(\tau; \theta)$ can be rewritten as the product of the Markov Process (i.e. the actions and transition probabilities)

\begin{aligned}
    P(\tau; \theta) &= \rho(s_0) \prod_{t=0}^{T-1} \pi_{\theta}(a_t | s_t) P(s_{t+1} | s_t, a_t) \\
    &\text{where} \\
    &\quad \rho(s_0) = \text{ Probability of starting at state } s_0 \\
    &\quad \pi_{\theta}(a_t | s_t) = \text{ Probability of taking action } a \text{ at state } s 
    \text{ under policy } \pi_{\theta} \\
    &\quad P(s_{t+1} | s_t, a_t) = \text{ Transition probability } \\
\end{aligned}

- Therefore, $\bigtriangledown_{\theta} J(\theta)$ can be written as

\begin{aligned}
    \bigtriangledown_{\theta} J(\theta) &= \bigtriangledown_{\theta} \sum_{\tau} P(\tau; \theta) R(\tau) \\
    &= \sum_{\tau} \bigtriangledown_{\theta} P(\tau, \theta) R(\tau)
\end{aligned}

- By the **log-derivative trick**, we know that:

\begin{aligned}
    \bigtriangledown_{\theta} \log f(\theta) &= \frac{\bigtriangledown_{\theta} f(\theta)}{f(\theta)} \\
    \therefore f(\theta) &= f(\theta) \bigtriangledown_{\theta} \log f(\theta)
\end{aligned}

- Therefore, it must be true that

\begin{aligned}
    \bigtriangledown_{\theta} J(\theta) &= \bigtriangledown_{\theta} \sum_{\tau} P(\tau; \theta) R(\tau) \\
    &= \sum_{\tau} \bigtriangledown_{\theta} P(\tau, \theta) R(\tau) \\
    &= \sum_{\tau} P(\tau, \theta) \bigtriangledown_{\theta} \log P(\tau, \theta) R(\tau) \\
    &= \mathbb{E}_{\tau \sim P(\tau, \theta)}[R(\tau) \bigtriangledown_{\theta} \log P(\tau, \theta)]
\end{aligned}

- This suggests that the gradient of the reward $\bigtriangledown_{\theta}J(\theta)$ is simply the expected product of reward from following trajectory $R(\tau)$, multiplied by the derivative of log probability

- Ok, but how do we compute $\bigtriangledown_{\theta} \log P(\tau, \theta)$?

\begin{aligned}
    P(\tau; \theta) &= \rho(s_0) \prod_{t=0}^{T-1} \pi_{\theta}(a_t | s_t) P(s_{t+1} | s_t, a_t) \\
    \log P(\tau; \theta) &= \log [\rho(s_0) \prod_{t=0}^{T-1} \pi_{\theta}(a_t | s_t) P(s_{t+1} | s_t, a_t)] \\
    \log P(\tau; \theta) &= \log \rho(s_0) + \sum_{t=0}^{T-1} \log \pi_{\theta}(a_t | s_t) + \sum_{t=0}^{T-1} \log P(s_{t+1} | s_t, a_t) \\
\end{aligned}

- But notice that of all the terms in the last expression, only $\sum_{t=0}^{T-1} \log \pi_{\theta}(a_t | s_t)$ contains $\theta$! Therefore;

\begin{aligned}
    \bigtriangledown_{\theta} \log P(\tau; \theta) &= \bigtriangledown_{\theta} [\log \rho(s_0) + \sum_{t=0}^{T-1} \log \pi_{\theta}(a_t | s_t) + \sum_{t=0}^{T-1} \log P(s_{t+1} | s_t, a_t)] \\
    &= \bigtriangledown_{\theta} \sum_{t=0}^{T-1} \log \pi_{\theta}(a_t | s_t) \\
\end{aligned}



- Substituting back into our original gradient expression

\begin{aligned}
    \bigtriangledown_{\theta} J(\theta) &= \mathbb{E}_{\tau \sim P(\tau, \theta)}[R(\tau) \bigtriangledown_{\theta} \log P(\tau, \theta)] \\
    &= \mathbb{E}_{\tau \sim P(\tau, \theta)}[R(\tau) \bigtriangledown_{\theta} \sum_{t=0}^{T-1} \log \pi_{\theta}(a_t | s_t)]
\end{aligned}

- We usually subsitute the reward $R(\tau)$ with a manageable number of discounted reward steps, giving us th final expression:

\begin{aligned}
    \bigtriangledown_{\theta} J(\theta) &= \mathbb{E}_{\tau \sim P(\tau, \theta)}[R(\tau) \bigtriangledown_{\theta} \sum_{t=0}^{T-1} \log \pi_{\theta}(a_t | s_t)] \\
    &= \mathbb{E}_{\tau \sim P(\tau, \theta)}[\sum_{i=0}^{k} \gamma^i r_{t+i} \bigtriangledown_{\theta} \sum_{t=0}^{T-1} \log \pi_{\theta}(a_t | s_t)] \\
    &= \mathbb{E}_{\tau \sim P(\tau, \theta)} [G_t \bigtriangledown_{\theta} \sum_{t=0}^{T-1} \log \pi_{\theta}(a_t | s_t)]
\end{aligned}

- This is why, for each iteration, we update $\theta$ by taking

\begin{aligned}
    \theta \leftarrow \theta + \alpha \cdot \bigtriangledown_{\theta} \sum_{t=0}^{T-1} \log \pi_{\theta}(a_t | s_t) \cdot G_t
\end{aligned}

- Intuitively, the gradient moves us in the direction of improving $J(\theta)$, with $\alpha$ as the learning rate controlling step size

- Note that here, assuming an episodic task, $G_t$ is simply the discounted reward of all steps in the episode remaining after time $t$

### Theory: Choosing a differentiable policy $\pi_{\theta}$

- We established the update step above as 

\begin{aligned}
    \theta \leftarrow \theta + \alpha \cdot \bigtriangledown_{\theta} \sum_{t=0}^{T-1} \log \pi_{\theta}(a_t | s_t) \cdot G_t
\end{aligned}

- So the core idea here is: we need some $\pi_{\theta}$ to be **differentiable w.r.t its parameters**, because you need to compute $\bigtriangledown_{\theta} \log \pi_{\theta}(a | s)$

- That is, the probability of sampling action a $P(a)$ must be a function of $\theta$, so we can compute $\frac{\partial P(a)}{\partial \theta}$

- Here, we will go through a few examples of what forms $P(a)$ can take, so that we can differentiate the policies for both continuous and discrete action spaces 

#### Softmax Policy

#### Gaussian Policy with Direct Parameter Update

#### Gaussian Policy with Neural Network

#### Squashed Gaussian / Tanh-Gaussian Policy

- For bounded continuous actions, you can sample a Gaussian and squash with a tanh:

#### Distributions other than Gaussian/Softmax

- Continuous
    - Beta: $\alpha_{\theta}(s), \beta_{\theta}(s)$
        - Naturally bounded (0,1); can scale to arbitrary ranges

    - Laplace / Logistic: Mean, scale
        - heavy-tailed action distributions

    - Gaussian Mixture Model: Multiple gaussians + their mixture weights
        - Multimodal actions; useful when one Gaussian cannot model the distribution

- Discrete
    - Gumbel-Softmax: logits + Gumbel noise + temperature
        - Used when you need the sampling process itself to be differentiable (i.e. not just argmax of the action-values)
        - Rare in RL

- Remember the core idea: any distribution that is **differentiable w.r.t its parameters** works, because you need to compute $\bigtriangledown_{\theta} \log \pi_{\theta}(a | s)$
    - That is, $\log P(a)$ must be a function of $\theta$, so we can compute $\frac{\partial \log P(a)}{\partial \theta}$

## 5. Actor–Critic Overview

- Combines **value-based** and **policy-based** methods:
  - **Actor**: updates the policy (like policy gradient).  
  - **Critic**: estimates value function (like TD) to reduce variance of updates.
- Intuition: the critic **guides** the actor by telling it whether actions are good or bad.

---

## 6. Key Takeaways

| Method | Model Needed | Online Learning | Bias vs Variance | Action Space |
|--------|--------------|----------------|----------------|--------------|
| Dynamic Programming | Yes | No | None | Discrete |
| Monte Carlo | No | Episodic | Low variance, high bias? | Discrete/Continuous |
| Temporal-Difference | No | Yes | Some bias, lower variance | Discrete/Continuous |
| Policy Gradient | No | Yes | High variance | Continuous-friendly |
| Actor–Critic | No | Yes | Balanced bias/variance | Continuous-friendly |

- Choice depends on **model availability**, **task type**, and **action space**.
- These methods form the backbone for **practical RL algorithms** like DQN, PPO, A3C, etc.

---

*Next: Practical Implementation — coding examples, environments, and debugging tips.*
